# Imports

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from utils import data_prep_utils, general_utils
from task_one import task_one_models
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [2]:
beat_rnaseq = data_prep_utils.gene_exp_log_trans(data_prep_utils.get_df("beat_rnaseq", nb=True))
beat_drug_df = data_prep_utils.ic50_log_trans(data_prep_utils.get_data_reorgenized(beat_rnaseq, data_prep_utils.missing_vals_knn(data_prep_utils.get_df("beat_drug", nb=True))))

In [4]:
beat_rnaseq.head()

,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,RP11-218F10.3,RP11-686O6.2,RP11-313P22.1,RP11-548H3.1,RP11-102N12.3,RP11-196O16.1,RP4-671G15.2,RP11-731C17.2,RP4-621B10.8,RP11-114I8.4
12-00066,3.332023,5.057945,4.359821,4.401159,9.840452,3.881618,5.750857,5.763965,6.075446,3.825562,...,3.391150,3.342047,3.352002,3.524769,3.352002,3.366807,3.337043,3.498121,3.326984,3.400773
12-00150,3.549603,5.397044,4.795890,4.409658,5.882753,4.190803,6.563612,5.624863,5.394115,4.336267,...,3.342335,3.409636,3.342335,3.499602,3.521243,3.470232,3.321928,3.409636,3.321928,3.346382
12-00211,3.321928,5.134117,5.574689,4.494539,6.928784,3.599120,6.110158,6.273325,6.450729,4.151148,...,3.525205,3.364939,3.341203,3.591092,3.331598,3.456163,3.331598,3.657950,3.374326,3.537790
12-00258,3.321928,5.298353,4.680766,4.185107,9.986494,5.336403,5.322683,5.755197,5.947014,3.910287,...,3.362726,3.436775,3.340203,3.660513,3.402401,3.371637,3.331094,3.453660,3.376073,3.353759
12-00294,3.321928,5.365999,5.620293,4.851765,8.118177,3.630876,5.564542,6.053756,6.130535,4.033776,...,3.580480,3.349971,3.354592,3.611700,3.382012,3.326640,3.354592,3.878789,3.435338,3.444037


In [5]:
beat_drug_df.head()

,A-674563,Afatinib (BIBW-2992),Alisertib (MLN8237),Axitinib (AG-013736),AZD1480,Barasertib (AZD1152-HQPA),BEZ235,BMS-345541,Bortezomib (Velcade),Bosutinib (SKI-606),...,TG100-115,Tofacitinib (CP-690550),Tozasertib (VX-680),Trametinib (GSK1120212),Vandetanib (ZD6474),Vargetef,Vatalanib (PTK787),Vismodegib (GDC-0449),VX-745,YM-155
12-00066,-0.519543,-0.567290,0.360390,-0.861206,0.385456,1.0,-1.417073,0.095660,-0.744268,-0.487401,...,0.502356,0.869371,0.296700,-1.862741,0.216112,-1.333854,0.088790,0.936307,-1.106552,-0.661768
12-00150,-0.736191,-1.021795,1.000000,1.000000,1.000000,1.0,1.000000,-0.102270,-0.996701,-0.155247,...,1.000000,1.000000,1.000000,1.000000,0.530730,-0.051009,0.409831,1.000000,1.000000,-0.197125
12-00211,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.888241,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
12-00258,0.352329,0.018141,1.000000,1.000000,1.000000,1.0,0.511842,0.515246,0.310652,1.000000,...,1.000000,-1.862741,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
12-00294,-0.505703,-0.384994,0.238384,-0.906013,1.000000,1.0,-1.597193,-0.022079,-0.317022,-0.532711,...,1.000000,0.245287,-0.136855,-1.862741,0.858581,-0.771062,0.588563,1.000000,-0.256898,-0.009582


# Test get_linear_reg_model

In [8]:
x_train, x_test, y_train, y_test = train_test_split(beat_rnaseq, beat_drug_df, test_size = 0.2, random_state = 42)

In [9]:
y_pred = task_one_models.get_linear_reg_model(x_train, y_train).predict(x_test)
testings_utils.get_mse(y_test.transpose(), y_pred.transpose())


0.4981080950441347

# Test get_gradient_boosting_tree_model

In [10]:
x_train, x_test, y_train, y_test = train_test_split(beat_rnaseq.iloc[:, 0:1000], beat_drug_df, test_size = 0.2, random_state = 42)
y_pred_2 = task_one_models.get_gradient_boosting_tree_model(x_train, y_train).predict(x_test)

KeyboardInterrupt: 

In [26]:
testings_utils.get_mse(y_test.transpose(), y_pred_2.transpose())

12.12263203722905

# Test get_reg_chain_model

In [11]:
y_pred_3 = task_one_models.get_reg_chain_model(x_train, y_train).predict(x_test)

In [12]:
testings_utils.get_mse(y_test.transpose(), y_pred_3.transpose())

0.6775306488418382

# Test get_lasso_reg_model

In [13]:
y_pred_4 = task_one_models.get_lasso_reg_model(x_train, y_train, 0.7, True).predict(x_test)

C:\Users\yaelz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


In [14]:
testings_utils.get_mse(y_test.transpose(), y_pred_4.transpose())

0.4607072583662443

# Test get_decision_tree_model

In [15]:
y_pred_5 = task_one_models.get_decision_tree_model(x_train, y_train).predict(x_test)

In [16]:
general_utils.get_mse(y_test.transpose(), y_pred_5.transpose())

0.988869889735281

# Test plot_kmeans_elbow

In [7]:
drug_not_t = beat_drug_df.transpose()
task_one_models.plot_kmeans_elbow(beat_drug_df, max_k=10)

In [14]:
drug_not_t = beat_drug_df.transpose()
kmenas = KMeans(n_clusters=4).fit(drug_not_t)
print(kmenas.labels_)
for label in range(4):
    print(f"{label}: {kmenas.labels_.tolist().count(label)}")
print(kmenas.predict(drug_not_t))
drug_not_t["cluster"] = kmenas.labels_
drug_not_t.head()
# print(max(kmenas.labels_))

[1 1 2 0 2 2 1 2 1 0 2 2 2 0 0 0 0 3 0 1 2 1 3 2 2 2 2 2 0 2 3 0 2 2 0 2 0
 2 0 2 0 0 2 2 2 1 2 0 2 0 1 2 1 3 1 0 1 1 0 0 2 2 2 0 2 3 0 2 3 2 2 2 3 2
 0 2 2 0 1]
0: 23
1: 13
2: 36
3: 7
[1 1 2 0 2 2 1 2 1 0 2 2 2 0 0 0 0 3 0 1 2 1 3 2 2 2 2 2 0 2 3 0 2 2 0 2 0
 2 0 2 0 0 2 2 2 1 2 0 2 0 1 2 1 3 1 0 1 1 0 0 2 2 2 0 2 3 0 2 3 2 2 2 3 2
 0 2 2 0 1]


,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270,cluster
A-674563,-0.519543,-0.736191,1.0,0.352329,-0.505703,0.046356,0.568114,0.182604,0.710352,0.264787,...,-0.342053,-0.983654,-0.714687,-0.288008,-0.274779,0.101013,-0.171291,-0.285657,-0.585897,1
Afatinib (BIBW-2992),-0.567290,-1.021795,1.0,0.018141,-0.384994,-0.307866,0.051799,-0.503998,0.647597,0.038274,...,0.019978,-0.254167,-0.210577,-0.340783,-0.091849,0.106886,-0.007327,0.304714,0.114974,1
Alisertib (MLN8237),0.360390,1.000000,1.0,1.000000,0.238384,0.788811,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.318231,0.950761,0.840545,0.974296,0.679946,1.000000,1.000000,1.000000,2
Axitinib (AG-013736),-0.861206,1.000000,1.0,1.000000,-0.906013,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,-0.378696,1.000000,1.000000,1.000000,1.000000,-0.524292,0
AZD1480,0.385456,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.837220,1.000000,1.000000,0.557230,0.845351,1.000000,1.000000,1.000000,1.000000,2


# Test split_drugs_by_kmeans

In [6]:
drug_not_t = beat_drug_df.transpose()
drug_not_t.head()

clusters = task_one_models.split_drugs_by_kmeans(beat_drug_df)
for cluster in clusters:
    print(cluster.shape)
clusters[0].head()

(198, 37)
(198, 10)
(198, 10)
(198, 22)


,Alisertib (MLN8237),AZD1480,Barasertib (AZD1152-HQPA),BMS-345541,Canertinib (CI-1033),Cediranib (AZD2171),CHIR-99021,Crenolanib,Erlotinib,GDC-0879,...,Ruxolitinib (INCB018424),SB-431542,SGX-523,STO609,TG100-115,Tofacitinib (CP-690550),Tozasertib (VX-680),Vandetanib (ZD6474),Vatalanib (PTK787),Vismodegib (GDC-0449)
12-00066,0.360390,0.385456,1.0,0.095660,1.000000,-0.903657,0.906307,-0.913968,1.000000,1.0,...,0.906307,1.000000,1.0,1.0,0.502356,0.869371,0.296700,0.216112,0.088790,0.936307
12-00150,1.000000,1.000000,1.0,-0.102270,0.257312,1.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,0.530730,0.409831,1.000000
12-00211,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.796302,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
12-00258,1.000000,1.000000,1.0,0.515246,0.701875,0.813349,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.0,1.0,1.000000,-1.862741,1.000000,1.000000,1.000000,1.000000
12-00294,0.238384,1.000000,1.0,-0.022079,0.394670,0.352223,1.000000,-0.204812,0.516680,1.0,...,1.000000,0.614614,1.0,1.0,1.000000,0.245287,-0.136855,0.858581,0.588563,1.000000
